In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [ ]:
# 시작전 설정
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline  
    # 주피터 노트북을 실행한 브라우저에서 바로 그림을 볼 수 있게 해줌. 안해도 요즘은 나온다
%config InlineBackend.figure_format='retina'  
    # 그래프를 더 높은 해상도로 그려줌
#한글설정
plt.rc('font', family='Malgun Gothic') # 윈도우즈
#plt.rc('font', family='AppleGothic') # mac
plt.rc('axes', unicode_minus=False)  # 축의 - 깨짐 방지
# 경고 메세지 안보이게
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 전처리

In [ ]:
# 사전설정 # 아이템 및 파일 경로 확인
import pandas as pd
import numpy as np

item = '배추'
item_code = 1001
df_path = f'datasets/작물_lag_월단위_스케일링X_인코딩X/{item}_월차낼게요.csv'
external_path = 'datasets/factor_external_weekly_ver_0721.csv'
save_path = f'datasets/작물_lag_월단위_스케일링X_인코딩X/{item}_월차_집계가공.csv'  # 중간저장 
# 양파 : 1201 # 배추 : 1001 # 상추 : 1005 # 사과 : 0601 # 무 : 1101 # 감자 : 0501 # 대파 : 1202 # 건고추 : 1207
# 마늘 : 1209 # 딸기 : 0804  # 방울토마토 : 0806 # 오이 : 0901 # 양배추 : 1004  # 고구마 : 0502  # 배 : 0602

In [ ]:
import pandas as pd

# 데이터 경로 설정
df_path = 'your_original_data_path.csv'
external_path = 'your_external_data_path.csv'
save_path = 'your_processed_data_path.csv'
item_code = 'your_item_code'

# 원본 데이터 로드
try:
    df = pd.read_csv(df_path, encoding='cp949')
except FileNotFoundError:
    print(f"오류: {df_path} 파일을 찾을 수 없습니다.")
    exit()

# '등급코드' 컬럼 제외
df_processed = df.drop(columns=['등급코드'])

# 주간 단위로 데이터 그룹화 및 집계
group_keys = ['year', 'week', '품종코드', '직팜산지코드']
agg_rules = {
    '평균단가(원)': 'mean',
    '총거래량(kg)': 'sum',
    '일평균기온_t-1': 'mean', '일평균기온_t-2': 'mean', '일평균기온_t-3': 'mean',
    '강수량(mm)_t-1': 'mean', '강수량(mm)_t-2': 'mean', '강수량(mm)_t-3': 'mean',
    '최고기온_t-1': 'mean', '최고기온_t-2': 'mean', '최고기온_t-3': 'mean',
    '최저기온_t-1': 'mean', '최저기온_t-2': 'mean', '최저기온_t-3': 'mean',
    '1시간최고강수량(mm)_t-1': 'mean', '1시간최고강수량(mm)_t-2': 'mean', '1시간최고강수량(mm)_t-3': 'mean',
    '평균상대습도_t-1': 'mean', '평균상대습도_t-2': 'mean', '평균상대습도_t-3': 'mean'
}

df_agg = df_processed.groupby(group_keys).agg(agg_rules).reset_index()
df_agg.drop_duplicates(inplace=True)

# 외생변수 데이터 로드
df_grow = pd.read_csv(external_path, encoding='utf-8')

# 'weekno'를 'year'와 'week'로 분리
df_grow['year'] = df_grow['weekno'] // 100
df_grow['week'] = df_grow['weekno'] % 100
df_grow.drop(columns='weekno', inplace=True)
             
# 병합을 위한 item_code 추가
df_agg['item_code'] = item_code

# 병합 전 불필요한 컬럼 삭제
df_agg.drop(columns=['holiday_flag', 'holiday_score', 'grow_score'], inplace=True, errors='ignore')

# 외생변수 병합
df_agg = pd.merge(
    df_agg,
    df_grow[['year', 'week', 'item_code', 'holiday_flag', 'holiday_score', 'grow_score']],
    on=['year', 'week', 'item_code'],
    how='left'
)

# 병합 후 'item_code' 삭제
df = df_agg.drop(columns='item_code')

# 최종 데이터를 CSV로 저장
df.to_csv(save_path, index=False)

# 최종 데이터 확인
print("데이터 전처리 및 병합 완료")
print(f"최종 데이터 크기: {df.shape[0]}행")
print("--- 최종 데이터 미리보기 ---")
print(df.head())

# 프로펫 + LSTM 모델 - Robust

In [ ]:
# 위 전처리 없이 아래만 따로 시작할때만 설정
# 양파 : 1201 # 배추 : 1001 # 상추 : 1005 # 사과 : 0601 # 무 : 1101 # 감자 : 0501 # 대파 : 1202 # 건고추 : 1207
# 마늘 : 1209 # 딸기 : 0804  # 방울토마토 : 0806 # 오이 : 0901 # 양배추 : 1004  # 고구마 : 0502  # 배 : 0602
ITEM = item
mid_save_path = f'datasets/작물_lag_월단위_스케일링X_인코딩X/{item}_월차_모델직전.csv'  # 중간저장 
model_path = 'final_model_checkpoints'

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib

# 기본 설정
TARGET_COL = '평균단가(원)'
TIMESTEPS = 8
LSTM_UNITS = 128
activation = 'relu'
epochs = 100
batch_size = 64
stop_patience = 40

# 데이터 로드
df = pd.read_csv(file_path)

# 그룹별 시계열 피처 생성
group_cols = ['직팜산지코드', '품종코드']
ts_feature_names = ['가격변화율', '가격차분']

df['가격변화율'] = df.groupby(group_cols)[TARGET_COL].pct_change().shift(1)
df['가격차분'] = df.groupby(group_cols)[TARGET_COL].diff().shift(1)

df[ts_feature_names] = df.groupby(group_cols)[ts_feature_names].fillna(0)
df.replace([np.inf, -np.inf], 0, inplace=True)

# 전국 단위로 데이터 집계
df['price_volume'] = df['평균단가(원)'] * df['총거래량(kg)']
agg_rules = {col: 'mean' for col in ts_feature_names}
agg_rules['price_volume'] = 'sum'
agg_rules['총거래량(kg)'] = 'sum'

df_agg = df.groupby(['year', 'week']).agg(agg_rules).reset_index()
df_agg[TARGET_COL] = df_agg['price_volume'] / df_agg['총거래량(kg)']
df_agg.drop(columns=['price_volume'], inplace=True)

# 중간 저장 (경로 확인)
df_agg.to_csv(mid_save_path, index=False)

# Prophet 피처 생성
df_prophet = df_agg.copy()
df_prophet['ds'] = pd.to_datetime(df_agg['year'].astype(str) + df_agg['week'].astype(str) + '0', format='%Y%W%w')
df_prophet['y'] = np.log1p(df_agg[TARGET_COL])

# Prophet 모델 학습 (훈련 데이터만 사용)
train_prophet_df = df_prophet[df_prophet['ds'].dt.year < 2025].copy()
regressor_cols = [col for col in agg_rules.keys() if col not in ['price_volume', '총거래량(kg)']]

prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
for col in regressor_cols:
    prophet_model.add_regressor(col)
prophet_model.fit(train_prophet_df[['ds', 'y'] + regressor_cols])

# 전체 기간에 대한 Prophet 예측 및 잔차 계산
forecast = prophet_model.predict(df_prophet[['ds'] + regressor_cols])
df_prophet['prophet_pred'] = forecast['yhat'].values
df_prophet['residual'] = df_prophet['y'] - df_prophet['prophet_pred']

# 미래 시점 예측을 위해 Target 변수 shift
df_prophet['y_target_-4d'] = df_prophet[TARGET_COL].shift(-4)
df_prophet = df_prophet.dropna()

# LSTM 학습용 데이터 분리
lstm_feature_cols = ['residual'] + regressor_cols
lstm_target_col = 'residual'

# 훈련/테스트 데이터 분리
cutoff_date = pd.to_datetime('2024-40-0', format='%Y-%W-%w')
train_final_df = df_prophet[df_prophet['ds'] <= cutoff_date].copy()
test_final_df = df_prophet[df_prophet['ds'] > cutoff_date].copy()

# 스케일링
scaler = RobustScaler()
train_final_df[lstm_feature_cols] = scaler.fit_transform(train_final_df[lstm_feature_cols])
test_final_df[lstm_feature_cols] = scaler.transform(test_final_df[lstm_feature_cols])

# 시퀀스 데이터 생성
def create_multivariate_sequences(df, feature_cols, target_col, timesteps):
    X, y = [], []
    features = df[feature_cols].values
    target = df[target_col].values
    for i in range(len(df) - timesteps):
        X.append(features[i:(i + timesteps)])
        y.append(target[i + timesteps])
    return np.array(X), np.array(y)

X_train_seq, y_train_seq = create_multivariate_sequences(train_final_df, lstm_feature_cols, lstm_target_col, TIMESTEPS)
X_test_seq, y_test_seq = create_multivariate_sequences(test_final_df, lstm_feature_cols, lstm_target_col, TIMESTEPS)

# LSTM 모델 구축 및 학습
lstm_model = Sequential([
    LSTM(units=LSTM_UNITS, activation=activation, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
    Dropout(0.2),
    Dense(units=1)
])
lstm_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# 콜백 설정
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_nm = f'best_model_LSTM+Prophet_{item}_{current_time}.keras'
checkpoint_filepath = os.path.join(model_path, model_nm)
checkpoint_cb = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=stop_patience, restore_best_weights=True, verbose=1)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6, verbose=1)

# 모델 학습
history = lstm_model.fit(X_train_seq, y_train_seq,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_test_seq, y_test_seq),
                        verbose=1,
                        callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb]
                       )

# 모델 및 Scaler 저장
# 저장 폴더 생성 (폴더 미존재 시)
if not os.path.exists('final_model_checkpoints'):
    os.makedirs('final_model_checkpoints')

# Prophet 모델 저장
joblib.dump(prophet_model, f'final_model_checkpoints/prophet_model_{item}.pkl')

# RobustScaler 저장
joblib.dump(scaler, f'final_model_checkpoints/robust_scaler_{item}.pkl')

# 모델 평가 및 시각화

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.models import load_model

# 최종 예측 및 평가
# 잔차 예측 결과 스케일링 복원
residual_pred_scaled = lstm_model.predict(X_test_seq)
dummy_array = np.zeros((len(residual_pred_scaled), len(lstm_feature_cols)))
dummy_array[:, 0] = residual_pred_scaled.flatten()
residual_pred = scaler.inverse_transform(dummy_array)[:, 0]

# 최종 예측값 계산 (Prophet 예측값 + LSTM 잔차 예측값)
prophet_future_pred = test_final_df['prophet_pred'].values[TIMESTEPS:]
final_pred_log = prophet_future_pred + residual_pred

# 로그 변환 복원
y_original_eval = test_final_df['y_original'].values[TIMESTEPS:]
final_pred_eval = np.expm1(final_pred_log)

# 모든 평가지표 계산
mae = mean_absolute_error(y_original_eval, final_pred_eval)
rmse = mean_squared_error(y_original_eval, final_pred_eval, squared=False)
mape = mean_absolute_percentage_error(y_original_eval, final_pred_eval) * 100
nmae = (mae / np.mean(y_original_eval)) * 100
r2 = r2_score(y_original_eval, final_pred_eval)

# 평가지표를 포함한 파일명으로 최종 모델 저장
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_filename = f'LSTM+Prophet_{item}_MAE_{mae:.0f}_R2_{r2:.4f}_{current_time}.keras'
final_model_path = os.path.join(model_path, final_model_filename)
lstm_model.save(final_model_path)

# 결과 출력
print("Prophet + LSTM 하이브리드 모델 최종 평가 결과")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"NMAE: {nmae:.2f}%")
print(f"R²: {r2:.4f}")

# 시각화
results_df = test_final_df.iloc[TIMESTEPS:].copy()
results_df['final_pred'] = final_pred_eval
results_df = results_df[results_df['year']==2025]

plt.figure(figsize=(16, 6))
plt.plot(results_df['ds'], results_df['y_original'], label='실제 평균단가', marker='o')
plt.plot(results_df['ds'], results_df['final_pred'], label='최종 예측단가', marker='x', linestyle='--')
plt.title(f"{test_final_df['ds'].dt.year.max()}년 주차별 평균단가 예측 결과 ({item})")
plt.xlabel("날짜")
plt.ylabel("평균단가(원)")
plt.legend()
plt.grid(True)
plt.show()

# 모델 로딩 및 예측값 출력

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
from datetime import datetime, timedelta
import os

# 기본 설정
MODEL_PATH = f'final_model_checkpoints/LSTM+Prophet_{ITEM}_MAE_111_R2_0.7894_20250726_025207.keras'
PROPHET_MODEL_PATH = f'final_model_checkpoints/prophet_model_{ITEM}.pkl'
SCALER_PATH = f'final_model_checkpoints/robust_scaler_{ITEM}.pkl'
DATA_PATH = mid_save_path
EXTERNAL_PATH = 'datasets/factor_external_weekly_ver_0721.csv'
OUTPUT_DIR = 'forecast_results'
TIMESTEPS = 8
FORECAST_WEEKS = 4

# 모델 및 데이터 로드
try:
    lstm_model = tf.keras.models.load_model(MODEL_PATH)
    prophet_model = joblib.load(PROPHET_MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    historical_df = pd.read_csv(DATA_PATH)
    external_df = pd.read_csv(EXTERNAL_PATH)
except FileNotFoundError as e:
    print(f"오류: 파일 로드 실패. {e}")
    exit()

def format_week_int(weekno):
    year = weekno // 100
    week = weekno % 100
    return year, week

# 외생변수 'weekno'를 'year'와 'week'로 분리
external_df[['year', 'week']] = external_df['weekno'].apply(
    lambda x: pd.Series(format_week_int(x))
)
external_df.drop(columns='weekno', inplace=True)

# 재귀 예측 함수
def recursive_forecast(lstm_model, prophet_model, scaler, historical_df, external_df, timesteps, forecast_weeks):
    # LSTM 피처 컬럼 (훈련 시점과 동일)
    lstm_feature_cols = ['residual'] + ['가격변화율', '가격차분']

    # Prophet 잔차 포함 데이터 생성
    historical_df['ds'] = pd.to_datetime(historical_df['year'].astype(str) + historical_df['week'].astype(str) + '0', format='%Y%W%w')
    prophet_full_pred = prophet_model.predict(historical_df)
    historical_df['residual'] = np.log1p(historical_df['평균단가(원)']) - prophet_full_pred['yhat']

    # 예측을 위한 초기 시퀀스 데이터 (최근 8주)
    last_date = historical_df['ds'].max()
    current_sequence = historical_df.tail(timesteps).copy()

    # 미래 외생변수는 가장 최근 값으로 유지
    last_external_features = current_sequence.tail(1)[['가격변화율', '가격차분', '총거래량(kg)']]

    predictions = []

    for i in range(forecast_weeks):
        # 현재 시퀀스로 LSTM 입력 데이터 생성
        lstm_input_features = current_sequence[lstm_feature_cols].values
        lstm_input_scaled = scaler.transform(lstm_input_features)
        lstm_input_reshaped = lstm_input_scaled.reshape(1, timesteps, len(lstm_feature_cols))

        # LSTM으로 다음 1주의 '잔차' 예측
        scaled_residual_pred = lstm_model.predict(lstm_input_reshaped, verbose=0)
        
        # 잔차 예측값 복원
        dummy_array = np.zeros((1, len(lstm_feature_cols)))
        dummy_array[:, 0] = scaled_residual_pred.flatten()
        predicted_residual = scaler.inverse_transform(dummy_array)[:, 0][0]

        # Prophet으로 다음 1주의 '기본 추세' 예측
        next_date = last_date + timedelta(weeks=i + 1)
        future_df = pd.DataFrame({'ds': [next_date]})
        
        # 미래 외생변수 채우기
        for col, val in last_external_features.iloc[0].items():
            future_df[col] = val
        
        prophet_baseline_pred = prophet_model.predict(future_df)['yhat'].iloc[0]

        # 최종 가격 예측
        final_log_pred = prophet_baseline_pred + predicted_residual
        final_price_pred = np.expm1(final_log_pred)
        predictions.append({'date': next_date, 'predicted_price': final_price_pred})

        # 다음 예측을 위해 시퀀스 업데이트
        new_row = pd.DataFrame([{'ds': next_date, 'residual': predicted_residual, **last_external_features.iloc[0].to_dict()}])
        current_sequence = pd.concat([current_sequence.iloc[1:], new_row], ignore_index=True)

    return pd.DataFrame(predictions)

# 예측 실행 및 CSV 저장
forecast_results_df = recursive_forecast(
    lstm_model, prophet_model, scaler, historical_df, external_df, TIMESTEPS, FORECAST_WEEKS
)

# CSV 파일로 저장
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

output_filename = os.path.join(OUTPUT_DIR, f'{ITEM}_price_forecast.csv')
forecast_results_df.to_csv(output_filename, index=False, encoding='utf-8')

# 결과 출력
print(f"향후 {FORECAST_WEEKS}주간 '{ITEM}' 가격 예측 결과")
print(forecast_results_df.to_string(index=False))

print(f"\n예측 결과를 '{output_filename}' 파일로 저장했습니다.")